**OLSQ DAC'22 tutorial**

It is recommended to pull OLSQ from [our GitHub repo](https://github.com/UCLA-VAST/OLSQ) becasue it is often more up-to-date, and there are more branches containing research progress.
However, we upload OLSQ to Python Package Index (`pip`) as well, which we use here.

In [1]:
!pip install OLSQ

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/58.4 MB ? eta -:--:--
   ---------------------------------------- 0.3/58.4 MB 9.9 MB/s eta 0:00:06
    --------------------------------------- 0.8/58.4 MB 12.7 MB/s eta 0:00:05
    --------------------------------------- 1.3/58.4 MB 12.0 MB/s eta 0:00:05
   - -------------------------------------- 1.9/58.4 MB 13.5 MB/s eta 0:00:05
   - -------------------------------------- 2.4/58.4 MB 14.1 MB/s eta 0:00:04
   - -------------------------------------- 2.9/58.4 MB 13.1 MB/s eta 0:00:05
   -- ------------------------------------- 3.4/58.4 MB 13.6 MB/s eta 0:00:05
   -- ------------------------------------- 3.9/58.4 MB 13.2 MB/s eta 0:00:05
   --- ------------------------------------ 4.4/58.4 MB 13.5 MB/s eta 0:00:04
   --- ------------------------------------ 5.1/58.4 MB 14.2 MB/s eta 0:00:04
   --- ------------------------------------ 5.7/5

The name of the package is `OLSQ`; the name of the module is `olsq`; and the name of the main solver class is `OLSQ`.
Additionally, we need to import a class `qcdevice` to load hardware information.

In [ ]:
from olsq import OLSQ 
from olsq.device import qcdevice

We initiate the solver with an **objective** and a **mode**.
The objective can be `depth`, `swap`, or `fidelity`.
The mode can be `normal` or `transition`.

In [3]:
solver = OLSQ("depth", "normal")

We construct a `qcdevice` object containing hardware information.
The arguments are: name, number of qubits (`nqubits`), list of edges between the qubits (`connection`), and duration of a SWAP gate (`swap_duration`).
In our case, the QPU has five qubits connected in a 'bowtie' pattern.
We assume a SWAP is decomposed to three CNOTs.

<img src="https://github.com/tbcdebug/OLSQ/blob/master/dac22/bowtie.png?raw=1" width="300">

In [4]:
bowtie_connections = [(0,1), (0,2), (1,2), (2,3), (2,4), (3,4)]
bowtie_dev = qcdevice("bowtie", nqubits=5, connection=bowtie_connections, 
                  swap_duration=3)

solver.setdevice(bowtie_dev)

We specify a program to compile with the intermediate representation of OLSQ consisting of three objects: number of physical qubits, a list of qubits involved in each gate (`gate_qubits`), and a list of specifications of these gates (`gate_name`).
In `gate_qubits`, every element is a tuple: if the gate is on one qubit, the tuple has only one elements, otherwise the tuple has two elements.
`gate_name` keeps all other information and is not involved in the solving process.

In [5]:
gate_qubits = [(0,), (1,), (3,), (2,3), (0,), (1,), (2,), (3,), (0,1), (2,3),
               (3,0), (1,2), (0,1), (2,3), (0,), (1,), (2,), (3,), (0,1), (2,3),
               (3,), (3,0), (3,)]

gate_name = ["x", "x", "h", "cnot", "t", "t", "t", "tdg", "cnot", "cnot", 
             "cnot", "cnot", "cnot", "cnot", "tdg", "tdg", "tdg", "t", "cnot",
             "cnot", "s", "cnot", "h"]

solver.setprogram([4, gate_qubits, gate_name], input_mode="IR")

The circuit diagram of this quantum program is shown below.
We can compare it with the output in the following cell.

<img src="https://github.com/tbcdebug/OLSQ/blob/master/dac22/adder.png?raw=1" width="750">

In [6]:
# print out the OLSQ IR
for i in range(len(gate_qubits)):
    if len(gate_qubits[i]) == 1:
        print(f"g{i} {gate_name[i]} q{gate_qubits[i][0]}")
    else:
        print(f"g{i} {gate_name[i]} q{gate_qubits[i][0]} q{gate_qubits[i][1]}")

g0 x q0
g1 x q1
g2 h q3
g3 cnot q2 q3
g4 t q0
g5 t q1
g6 t q2
g7 tdg q3
g8 cnot q0 q1
g9 cnot q2 q3
g10 cnot q3 q0
g11 cnot q1 q2
g12 cnot q0 q1
g13 cnot q2 q3
g14 tdg q0
g15 tdg q1
g16 tdg q2
g17 t q3
g18 cnot q0 q1
g19 cnot q2 q3
g20 s q3
g21 cnot q3 q0
g22 h q3


Solve the problem with `solve()` method.
It will try to increase the depth limit if no solution is found under the current limit.
The return value contains the five objects below.
There will also be some information about the results printed out.

In [7]:
result_depth, list_scheduled_gate_name, list_scheduled_gate_qubits,\
final_mapping, objective_value = solver.solve(output_mode="IR")

Trying maximal depth = 11...
Trying maximal depth = 14...
Trying maximal depth = 18...
Compilation time = 0:00:32.575355.
SWAP on physical edge (0,2) at time 7
Gate 0: x 0 on qubit 4 at time 0
Gate 1: x 1 on qubit 3 at time 0
Gate 2: h 3 on qubit 0 at time 0
Gate 3: cnot 2, 3 on qubits 2 and 0 at time 1
Gate 4: t 0 on qubit 4 at time 1
Gate 5: t 1 on qubit 3 at time 1
Gate 6: t 2 on qubit 2 at time 2
Gate 7: tdg 3 on qubit 0 at time 2
Gate 8: cnot 0, 1 on qubits 4 and 3 at time 2
Gate 9: cnot 2, 3 on qubits 2 and 0 at time 3
Gate 10: cnot 3, 0 on qubits 2 and 4 at time 8
Gate 11: cnot 1, 2 on qubits 3 and 2 at time 4
Gate 12: cnot 0, 1 on qubits 4 and 3 at time 9
Gate 13: cnot 2, 3 on qubits 0 and 2 at time 9
Gate 14: tdg 0 on qubit 4 at time 10
Gate 15: tdg 1 on qubit 3 at time 10
Gate 16: tdg 2 on qubit 0 at time 10
Gate 17: t 3 on qubit 2 at time 10
Gate 18: cnot 0, 1 on qubits 4 and 3 at time 11
Gate 19: cnot 2, 3 on qubits 0 and 2 at time 11
Gate 20: s 3 on qubit 2 at time 12
Gate

In [8]:
# making use of the return values to print circuit in a more friendly way
print(f"objective value: {objective_value}")
print("\nprogram:")
for t in range(result_depth):
    print(f"time {t}:")
    for i in range(len(list_scheduled_gate_name[t])):
        if len(list_scheduled_gate_qubits[t][i]) == 1:
            print(f"    {list_scheduled_gate_name[t][i]} " +
                  f"p{list_scheduled_gate_qubits[t][i][0]}")
        else:
            print(f"    {list_scheduled_gate_name[t][i]} " +
                  f"p{list_scheduled_gate_qubits[t][i][0]} " +
                  f"p{list_scheduled_gate_qubits[t][i][1]} ")
print("\nfinal_mapping")
for i, j in enumerate(final_mapping):
    print(f"q{i} -> p{j}")

objective value: 15

program:
time 0:
    x p4
    x p3
    h p0
time 1:
    cnot p2 p0 
    t p4
    t p3
time 2:
    t p2
    tdg p0
    cnot p4 p3 
time 3:
    cnot p2 p0 
time 4:
    cnot p3 p2 
time 5:
    cx p0 p2 
time 6:
    cx p2 p0 
time 7:
    cx p0 p2 
time 8:
    cnot p2 p4 
time 9:
    cnot p4 p3 
    cnot p0 p2 
time 10:
    tdg p4
    tdg p3
    tdg p0
    t p2
time 11:
    cnot p4 p3 
    cnot p0 p2 
time 12:
    s p2
time 13:
    cnot p2 p4 
time 14:
    h p2

final_mapping
q0 -> p4
q1 -> p3
q2 -> p0
q3 -> p2


In [9]:
# setting the objective to swap and try again
solver_swap = OLSQ("swap", "normal")
solver_swap.setdevice(bowtie_dev)
solver_swap.setprogram([4, gate_qubits, gate_name], input_mode="IR")
results_swap = solver_swap.solve(output_mode="IR")

Trying maximal depth = 11...
Trying maximal depth = 14...
Trying maximal depth = 18...
Compilation time = 0:00:35.313887.
SWAP on physical edge (0,2) at time 10
Gate 0: x 0 on qubit 3 at time 0
Gate 1: x 1 on qubit 4 at time 0
Gate 2: h 3 on qubit 0 at time 0
Gate 3: cnot 2, 3 on qubits 2 and 0 at time 1
Gate 4: t 0 on qubit 3 at time 1
Gate 5: t 1 on qubit 4 at time 1
Gate 6: t 2 on qubit 2 at time 2
Gate 7: tdg 3 on qubit 0 at time 2
Gate 8: cnot 0, 1 on qubits 3 and 4 at time 2
Gate 9: cnot 2, 3 on qubits 2 and 0 at time 4
Gate 10: cnot 3, 0 on qubits 2 and 3 at time 11
Gate 11: cnot 1, 2 on qubits 4 and 2 at time 6
Gate 12: cnot 0, 1 on qubits 3 and 4 at time 12
Gate 13: cnot 2, 3 on qubits 0 and 2 at time 12
Gate 14: tdg 0 on qubit 3 at time 13
Gate 15: tdg 1 on qubit 4 at time 13
Gate 16: tdg 2 on qubit 0 at time 13
Gate 17: t 3 on qubit 2 at time 13
Gate 18: cnot 0, 1 on qubits 3 and 4 at time 14
Gate 19: cnot 2, 3 on qubits 0 and 2 at time 14
Gate 20: s 3 on qubit 2 at time 15


In [10]:
# use the transition-based model, should run much faster
solver_transition = OLSQ("swap", "transition")
solver_transition.setdevice(bowtie_dev)
solver_transition.setprogram([4, gate_qubits, gate_name], input_mode="IR")
results_transition = solver_transition.solve(output_mode="IR")

Trying maximal depth = 1...
Trying maximal depth = 2...
Compilation time = 0:00:00.381027.
SWAP on physical edge (2,4) at time 0
Gate 0: x 0 on qubit 1 at time 0
Gate 1: x 1 on qubit 0 at time 0
Gate 2: h 3 on qubit 4 at time 0
Gate 3: cnot 2, 3 on qubits 2 and 4 at time 0
Gate 4: t 0 on qubit 1 at time 0
Gate 5: t 1 on qubit 0 at time 0
Gate 6: t 2 on qubit 2 at time 0
Gate 7: tdg 3 on qubit 4 at time 0
Gate 8: cnot 0, 1 on qubits 1 and 0 at time 0
Gate 9: cnot 2, 3 on qubits 2 and 4 at time 0
Gate 10: cnot 3, 0 on qubits 2 and 1 at time 1
Gate 11: cnot 1, 2 on qubits 0 and 2 at time 0
Gate 12: cnot 0, 1 on qubits 1 and 0 at time 1
Gate 13: cnot 2, 3 on qubits 4 and 2 at time 1
Gate 14: tdg 0 on qubit 1 at time 1
Gate 15: tdg 1 on qubit 0 at time 1
Gate 16: tdg 2 on qubit 4 at time 1
Gate 17: t 3 on qubit 2 at time 1
Gate 18: cnot 0, 1 on qubits 1 and 0 at time 1
Gate 19: cnot 2, 3 on qubits 4 and 2 at time 1
Gate 20: s 3 on qubit 2 at time 1
Gate 21: cnot 3, 0 on qubits 2 and 1 at ti

Let's try targetting a larger QPU based on heavy-hexagon lattice IBM implemented. 

<img src="https://github.com/tbcdebug/OLSQ/blob/master/dac22/heavyhex.png?raw=1" width="300">

In [11]:
# use the transition-based model, should run much faster
solver_transition = OLSQ("swap", "transition")
solver_transition.setdevice(qcdevice("heavy-hexagon", nqubits=18, 
                                     connection=[(0,4), (1,2), (2,3), (3,4),
                                                 (4,5), (5,6), (6,7), (2,8),
                                                 (6,9), (10,11), (8,11),
                                                 (11,12), (12,13), (13,14),
                                                 (14,15), (15,16), (9,15),
                                                 (13,17)], swap_duration=3))
solver_transition.setprogram([4, gate_qubits, gate_name], input_mode="IR")
results_transition = solver_transition.solve(output_mode="IR")

Trying maximal depth = 1...
Trying maximal depth = 2...
Compilation time = 0:00:02.588104.
SWAP on physical edge (0,4) at time 0
SWAP on physical edge (2,3) at time 0
Gate 0: x 0 on qubit 2 at time 0
Gate 1: x 1 on qubit 3 at time 0
Gate 2: h 3 on qubit 0 at time 0
Gate 3: cnot 2, 3 on qubits 4 and 0 at time 0
Gate 4: t 0 on qubit 2 at time 0
Gate 5: t 1 on qubit 3 at time 0
Gate 6: t 2 on qubit 4 at time 0
Gate 7: tdg 3 on qubit 0 at time 0
Gate 8: cnot 0, 1 on qubits 2 and 3 at time 0
Gate 9: cnot 2, 3 on qubits 4 and 0 at time 0
Gate 10: cnot 3, 0 on qubits 4 and 3 at time 1
Gate 11: cnot 1, 2 on qubits 3 and 4 at time 0
Gate 12: cnot 0, 1 on qubits 3 and 2 at time 1
Gate 13: cnot 2, 3 on qubits 0 and 4 at time 1
Gate 14: tdg 0 on qubit 3 at time 1
Gate 15: tdg 1 on qubit 2 at time 1
Gate 16: tdg 2 on qubit 0 at time 1
Gate 17: t 3 on qubit 4 at time 1
Gate 18: cnot 0, 1 on qubits 3 and 2 at time 1
Gate 19: cnot 2, 3 on qubits 0 and 4 at time 1
Gate 20: s 3 on qubit 4 at time 1
Gate